# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# List of all books
all_books = []

# Loop through the 50 pages
for i in range(1, 51):
    url = f"http://books.toscrape.com/catalogue/page-{i}.html"
    response = requests.get(url)
    
    # Audit : Polite scraper attention to the server
    time.sleep(0.5) 
    
    if response.status_code != 200:
        print(f"Page error {i}")
        continue
        
    soup = BeautifulSoup(response.content, "html.parser")
    books = soup.find_all("article", class_="product_pod")
    
    for book in books:
        # Self-contained extraction with error handling
        try:
            title = book.h3.a["title"]
            price_text = book.find("p", class_="price_color").text
            # Cleaning the price string and converting to float suppressing the currency symbol
            price = float(price_text.replace("£", ""))
            
            # Rating extraction
            rating = book.p["class"][1]
            
            # Availability extraction
            stock = book.find("p", class_="instock availability").text.strip()
            
            all_books.append({
                "Title": title,
                "Price_GBP": price,
                "Rating": rating,
                "Availability": stock,
                "Page_Source": i
            })
        except Exception as e:
            print(f"Erreur d'extraction sur un livre : {e}")

# Turn the list of books into a DataFrame
df_final = pd.DataFrame(all_books)

In [13]:
df_final.head()

,Title,Price_GBP,Rating,Availability,Page_Source
0,A Light in the Attic,51.77,Three,In stock,1
1,Tipping the Velvet,53.74,One,In stock,1
2,Soumission,50.10,One,In stock,1
3,Sharp Objects,47.82,Four,In stock,1
4,Sapiens: A Brief History of Humankind,54.23,Five,In stock,1


In order to compute statistics with the rating, turn it into number integer

In [18]:
rating_map = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5}
df_final['Rating_Num'] = df_final['Rating'].map(rating_map).astype(int)
df_final['Rating_Num'].dtype

dtype('int64')

In [19]:
# Au lieu de juste tester, affichez le résultat avec un message
count = len(df_final)
print(f"Audit de l'extraction : {count} livres récupérés.")

Audit de l'extraction : 1000 livres récupérés.


In [20]:
df_final.to_csv('results.csv', index=False)